In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO  # YOLOv8 모델 사용

# 1️⃣ YOLOv8 모델 로드 (사전 학습된 모델 사용)
model = YOLO("yolov8n.pt")  # YOLOv8 기본 모델 (n: nano, l: large 등)

# 2️⃣ 이미지 불러오기
image_path = "C:/Users/wlsth/OneDrive/Desktop/MS_Project/data/CrackedPhone/resized_crackedPhone(B)_R/crackedPhone1.jpg"
img = cv2.imread(image_path)

# 3️⃣ YOLO로 핸드폰 감지
results = model(img)

# 핸드폰 감지된 영역 저장
phone_detected = False
for r in results:
    for box in r.boxes:
        cls = int(box.cls[0])  # 감지된 객체의 클래스
        if model.names[cls] == "cell phone":  # YOLO 클래스 이름 확인
            phone_detected = True
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            
            # 4️⃣ 핸드폰 영역만 자르기
            phone_img = img[y1:y2, x1:x2]
            
            # 5️⃣ 크랙 분석을 위해 저장
            cropped_path = "cropped_phone.jpg"
            cv2.imwrite(cropped_path, phone_img)
            print(f"📱 핸드폰 감지 완료! 크롭된 이미지 저장: {cropped_path}")

            # 6️⃣ Canny 엣지 검출 (크랙 분석)
            gray = cv2.cvtColor(phone_img, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 100, 200)

            # 7️⃣ 크랙 영역 찾기 (Contours)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(phone_img, contours, -1, (0, 0, 255), 2)  # 윤곽선 빨간색

            # 8️⃣ 결과 출력
            print(f"🔍 검출된 크랙 개수: {len(contours)}")

            # 9️⃣ 결과 이미지 출력
            cv2.imshow("Phone Image", phone_img)
            cv2.imshow("Crack Detection", edges)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

if not phone_detected:
    print("⚠ 핸드폰을 감지하지 못했습니다. 이미지 확인 필요!")